In [ ]:
import pandas as pd
from IPython.display import display
import re


# $ gsutil ls -la "gs://durrant_prot_pred_db/mgnify_scrape_20220505/**" | tee full_scrape.txt
LS_FPATH = '/GeneGraphDB/data/full_scrape.txt'

In [ ]:
with open(LS_FPATH) as f:
    lines = f.readlines()

In [ ]:
lines[0:5]

In [ ]:
analysis_pattern = re.compile(r'\s*(\d+)\s+2022.*Z\s+gs.*studies\/(\w*)\/samples\/(\w*)/analyses\/(\w+)\/(.*)#.*$')


parsed = []
for line in lines:
    match_res = analysis_pattern.match(line)
    if match_res:
      
        parsed_dict = {
            'size': int(match_res.group(1)),
            'study_id': match_res.group(2),
            'sample_id': match_res.group(3),
            'analysis_id': match_res.group(4),
            'filename': match_res.group(5),
        }
        parsed.append(parsed_dict)
        
df = pd.DataFrame(parsed)
full_df = df.copy()

df = df[~df['filename'].str.contains('json')]
df = df[df['filename'].str.endswith('.faa.gz')]
# df = df[df['size'] > 10**6]


In [ ]:
studies_pattern = re.compile(r'\s*(\d+)\s+2022.*Z\s+gs.*studies\/(\w*).json')

In [ ]:
df.head()

In [ ]:
(df[['analysis_id', 'size']].groupby('analysis_id').sum() / 10**6).describe()

In [ ]:
df['size'].sum() / 10**9

In [ ]:
df.shape  # compared to 61k analysis.json files.  ~15% of studies have a CDS file?

In [ ]:
x = df.groupby('analysis_id').count()
x['size'].value_counts() / x['size'].index.size

In [ ]:
analysis_df = full_df[full_df['filename'].str.startswith('analysis') & full_df['filename'].str.endswith(".json")]
analysis_df.groupby('study_id').count()

In [ ]:
df.groupby('study_id').count()